# Beam example 
A simply suppoted beam subjected to a distributed load with constant intensity. 
It is modelled using 2D-beam elements (beam2e/beam2s) using four elements in series. 

Dofs and elements:
```
       (1)         (2)         (3)           (4)          
1,2,3 ----- 3,4,5 ----- 6,7,8 ----- 9,10,11 ----- 12,13,14
```

In [3]:
from mha021 import *

L = 1 # [m]
q = 10e3 # [N/m]
I = 1e-4 # [m^4]
E = 200e9 # [Pa]
A = 5e-3 # [m^2]

Ex = np.array([
    [0,  0.25*L],
    [0.25*L,  0.5*L],
    [0.5*L,  0.75*L],
    [0.75*L,  L]
])
Ey = np.array([ 
    [0,  0],
    [0,  0],
    [0,  0],
    [0,  0]
])

fig = draw_discrete_elements(
    Ex, Ey,
    title="Geometry with nodes and elements",
    xlabel="x [m]",
    ylabel="y [m]",
    annotate="both"
)
fig.show()

Edof = np.array([
    [1, 2, 3, 4, 5, 6],
    [4, 5, 6, 7, 8, 9],
    [7, 8, 9, 10, 11, 12],
    [10, 11, 12, 13, 14, 15]
])

num_el = Edof.shape[0]
qxy = np.zeros((num_el, 2)) # Distributed loads 
qxy[:, 1] = -q # All elements in vertical direction

ep = [E, A, I]
num_dofs = np.max(np.max(Edof))


In [ ]:
K = np.zeros((num_dofs, num_dofs)) # Stiffness matrix
f = np.zeros((num_dofs))        # Load vector

for el in range(num_el):
    dofs = Edof[el, :]   # DOFs for the element
    Ke, fe = beam2e(Ex[el, :], Ey[el, :], E, A, I, qxy[el, :])  # Element stiffness matrix
    assem(K, Ke, dofs)
    assem(f, fe, dofs)

# Boundary conditions
bc_dofs = [1, 2, 14]
bc_vals = [0.0, 0.0, 0.0]

# # Solve the system
a, r = solve_eq(K, f, bc_dofs, bc_vals)

displayvar("a", a, 2)
displayvar("r", r, 2)

## Postprocessing
Ed = extract_dofs(a, Edof)

# Draw the deformed structure
fig = draw_discrete_elements(Ex, Ey, title="Deformed frame", line_style="dashed")
plot_deformed_beams(fig, Ex, Ey, Ed, scale=1e4)
fig.show()

M = np.zeros((num_el, 2))
N = np.zeros((num_el, 2))
for el in range(num_el):
    data = beam2s(Ex[el, :], Ey[el, :], Ed[el, :], E, A, I)
    M[el, :] = data["M"]
    N[el, :] = data["N"]

displayvar("a_{8}", a[8-1]) 
δ = 5*q*L**4/(384*E*I) # beam bending case, simply supported subjected to distributed load
displayvar("δ", δ) 

displayvar("M", M) # note the error in bending moment at the ends, the beam is simply supported so the value should be zero
displayvar("N", N) 

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>